In [45]:
import pandas as pd
import numpy as np

In [46]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from mlxtend.classifier import StackingCVClassifier
from sklearn import model_selection

In [47]:

df = pd.read_csv('CIC2017.csv')
#df.replace([np.inf, -np.inf], df.mean())
df.head()
df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

#df.isnull().any()
#dtype={'Bwd Packet Length Std': float}
#dtype={'Flow Bytes/s': float}


c:\users\ismail\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,_Destination_Port,_Flow_Duration,_Total_Fwd_Packets,_Total_Backward_Packets,Total_Length_of_Fwd_Packets,_Total_Length_of_Bwd_Packets,_Fwd_Packet_Length_Max,_Fwd_Packet_Length_Min,_Fwd_Packet_Length_Mean,_Fwd_Packet_Length_Std,...,_min_seg_size_forward,Active_Mean,_Active_Std,_Active_Max,_Active_Min,Idle_Mean,_Idle_Std,_Idle_Max,_Idle_Min,_label
0,54865.0,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,55054.0,109.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,55055.0,52.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,46236.0,34.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,54863.0,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225741,61374.0,61.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
225742,61378.0,72.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
225743,61375.0,75.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
225744,61323.0,48.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [48]:
df = df.drop( df[ (df._label != "BENIGN") & (df._label != "DDoS") ].index )

print(df.shape)

(225745, 79)


In [49]:
df=df.sample(n=20000)
df.head()

,_Destination_Port,_Flow_Duration,_Total_Fwd_Packets,_Total_Backward_Packets,Total_Length_of_Fwd_Packets,_Total_Length_of_Bwd_Packets,_Fwd_Packet_Length_Max,_Fwd_Packet_Length_Min,_Fwd_Packet_Length_Mean,_Fwd_Packet_Length_Std,...,_min_seg_size_forward,Active_Mean,_Active_Std,_Active_Max,_Active_Min,Idle_Mean,_Idle_Std,_Idle_Max,_Idle_Min,_label
114504,80.0,1872604.0,3.0,4.0,26.0,11607.0,20.0,0.0,8.67,10.26,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
89876,80.0,188196.0,3.0,6.0,26.0,11601.0,20.0,0.0,8.67,10.26,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
134237,80.0,1152838.0,3.0,5.0,26.0,11601.0,20.0,0.0,8.67,10.26,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
156616,59741.0,97699013.0,5.0,9.0,12982.0,56.0,11595.0,0.0,2596.40,5065.28,...,20.0,3848255.0,0.0,3848255.0,3848255.0,93700000.0,0.0,93700000.0,93700000.0,BENIGN
66410,80.0,78494150.0,8.0,5.0,56.0,11607.0,20.0,0.0,7.00,5.66,...,20.0,993.0,0.0,993.0,993.0,39200000.0,44000000.0,70200000.0,8072789.0,DDoS


In [50]:
y = df['_label']
X = df.drop(['_label'], axis=1)


In [51]:
y = y.replace( to_replace =  'DDoS' , value = 1 )
y = y.replace( to_replace =  'BENIGN' , value = 0 )


In [52]:
X.head()

,_Destination_Port,_Flow_Duration,_Total_Fwd_Packets,_Total_Backward_Packets,Total_Length_of_Fwd_Packets,_Total_Length_of_Bwd_Packets,_Fwd_Packet_Length_Max,_Fwd_Packet_Length_Min,_Fwd_Packet_Length_Mean,_Fwd_Packet_Length_Std,...,_act_data_pkt_fwd,_min_seg_size_forward,Active_Mean,_Active_Std,_Active_Max,_Active_Min,Idle_Mean,_Idle_Std,_Idle_Max,_Idle_Min
114504,80.0,1872604.0,3.0,4.0,26.0,11607.0,20.0,0.0,8.67,10.26,...,2.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89876,80.0,188196.0,3.0,6.0,26.0,11601.0,20.0,0.0,8.67,10.26,...,2.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134237,80.0,1152838.0,3.0,5.0,26.0,11601.0,20.0,0.0,8.67,10.26,...,2.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156616,59741.0,97699013.0,5.0,9.0,12982.0,56.0,11595.0,0.0,2596.40,5065.28,...,3.0,20.0,3848255.0,0.0,3848255.0,3848255.0,93700000.0,0.0,93700000.0,93700000.0
66410,80.0,78494150.0,8.0,5.0,56.0,11607.0,20.0,0.0,7.00,5.66,...,6.0,20.0,993.0,0.0,993.0,993.0,39200000.0,44000000.0,70200000.0,8072789.0


In [53]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier,StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from vecstack import stacking

In [54]:
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score


scoring_metrics = { accuracy_score , precision_score , recall_score , f1_score }

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


## Adaboost+Decision Tree (base)+MLP(meta)+PCA

In [56]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from numpy import inf

# Create Base Learners
sc = StandardScaler()
#print(X_train)
X_train = np.array(X_train, dtype=float)
X_test = np.array(X_test, dtype=float)

X_train[X_train == inf] = 9999
X_test[X_test == inf] = 9999

#print(X_test)


#X_train = X_train.fillna(lambda x: x.median())
#X_train=np.nan_to_num(X_train)
#np.isinf(X_train).any()
#X_train.head()
#X_train=X_train.dropna()
#X_train = X_train.fillna(X_train.mean())

X_train = sc.fit_transform(X_train)
#X_train = SimpleImputer(missing_values=np.nan, strategy='mean')
X_test = sc.transform(X_test)
print(np.isnan(X_test).any())
from sklearn.decomposition import PCA
X_train=np.nan_to_num(X_train)
X_test=np.nan_to_num(X_test)

pca = PCA(n_components=6)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

False


In [13]:
from sklearn.tree import tree

# Create Base Learners
base_learners = [tree.DecisionTreeClassifier(max_depth= 13 , min_samples_split = 10 ) ,
                 AdaBoostClassifier(learning_rate=0.1 , n_estimators=1000)          
                ]
meta_learner = MLPClassifier(alpha=0.01 , hidden_layer_sizes=20 , learning_rate='constant' , solver='adam')

c:\users\ismail\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


In [14]:
# Initialize Stacking Classifier with the Meta Learner
S_train, S_test = stacking(base_learners,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
#                            mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric= accuracy_score , 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.99675000]
    fold  1:  [0.99800000]
    fold  2:  [0.99925000]
    fold  3:  [0.99625000]
    ----
    MEAN:     [0.99756250] + [0.00116425]
    FULL:     [0.99756250]

model  1:     [AdaBoostClassifier]
    fold  0:  [0.99825000]
    fold  1:  [0.99825000]
    fold  2:  [0.99750000]
    fold  3:  [0.99775000]
    ----
    MEAN:     [0.99793750] + [0.00032476]
    FULL:     [0.99793750]



In [15]:
model = meta_learner.fit(S_train, y_train)
y_pred = meta_learner.predict(S_test)
print('Accuracy score: [%.8f]' % accuracy_score(y_test, y_pred))
print('Precision score: [%.8f]' % precision_score(y_test, y_pred))
print('recall score: [%.8f]' % recall_score(y_test, y_pred))


Accuracy score: [0.99750000]
Precision score: [0.99779541]
recall score: [0.99779541]


In [28]:
# Initialize Stacking Classifier with the Meta Learner
S_train, S_test = stacking(base_learners,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
#                            mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric= accuracy_score , 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)stacking3 = { 'accuracy' : accuracy_score(y_test, y_pred) , 'Precision' : precision_score(y_test, y_pred) , 'recall' : recall_score(y_test, y_pred) }

stacking3 = pd.DataFrame(stacking3 , index=[0,1] )

SyntaxError: invalid syntax (<ipython-input-28-5d3b46719497>, line 22)

## Adaboost+KNN(base)+MLP(meta)+PCA

In [29]:
from sklearn.preprocessing import StandardScaler

# Create Base Learners
sc = StandardScaler() 
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from sklearn.decomposition import PCA

pca = PCA(n_components=6)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
base_learners = [  RandomForestClassifier(n_estimators=90, min_samples_split=0.1 , max_depth=19 ) ,
                    AdaBoostClassifier(learning_rate=0.1 , n_estimators=1000)      
                ]
meta_learner = MLPClassifier(alpha=0.01 , hidden_layer_sizes=20 , learning_rate='constant' , solver='adam')

In [30]:
# Initialize Stacking Classifier with the Meta Learner
S_train, S_test = stacking(base_learners,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
#                            mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric = accuracy_score , 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [RandomForestClassifier]
    fold  0:  [0.99700000]
    fold  1:  [0.99550000]
    fold  2:  [0.99825000]
    fold  3:  [0.99625000]
    ----
    MEAN:     [0.99675000] + [0.00101550]
    FULL:     [0.99675000]

model  1:     [AdaBoostClassifier]
    fold  0:  [0.99900000]
    fold  1:  [0.99825000]
    fold  2:  [0.99850000]
    fold  3:  [0.99725000]
    ----
    MEAN:     [0.99825000] + [0.00063738]
    FULL:     [0.99825000]



In [31]:
model = meta_learner.fit(S_train, y_train)
y_pred = meta_learner.predict(S_test)
print('Accuracy score: [%.8f]' % accuracy_score(y_test, y_pred))
print('Precision score: [%.8f]' % precision_score(y_test, y_pred))
print('recall score: [%.8f]' % recall_score(y_test, y_pred))


Accuracy score: [0.99825000]
Precision score: [1.00000000]
recall score: [0.99692982]


In [32]:

stacking1 = { 'accuracy' : accuracy_score(y_test, y_pred) , 'Precision' : precision_score(y_test, y_pred) , 'recall' : recall_score(y_test, y_pred) }

stacking1 = pd.DataFrame(stacking1 , index=[0,1] )

## Adaboost+RF(base)+MLP(meta)+PCA

In [57]:

from sklearn.decomposition import PCA

pca = PCA(n_components=6)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
base_learners = [  RandomForestClassifier(n_estimators=90, min_samples_split=0.1 , max_depth=19 ) ,
                    AdaBoostClassifier(learning_rate=0.1 , n_estimators=1000)      
                ]
meta_learner = MLPClassifier(alpha=0.01 , hidden_layer_sizes=20 , learning_rate='constant' , solver='adam')

# Initialize Stacking Classifier with the Meta Learner
S_train, S_test = stacking(base_learners,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
#                            mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric = accuracy_score , 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

model = meta_learner.fit(S_train, y_train)
y_pred = meta_learner.predict(S_test)
print('Accuracy score: [%.8f]' % accuracy_score(y_test, y_pred))
print('Precision score: [%.8f]' % precision_score(y_test, y_pred))
print('recall score: [%.8f]' % recall_score(y_test, y_pred))



stacking2 = { 'accuracy' : accuracy_score(y_test, y_pred) , 'Precision' : precision_score(y_test, y_pred) , 'recall' : recall_score(y_test, y_pred) }

stacking2 = pd.DataFrame(stacking2 , index=[0,1] )

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [RandomForestClassifier]
    fold  0:  [0.99825000]
    fold  1:  [0.99825000]
    fold  2:  [0.99875000]
    fold  3:  [0.99775000]
    ----
    MEAN:     [0.99825000] + [0.00035355]
    FULL:     [0.99825000]

model  1:     [AdaBoostClassifier]
    fold  0:  [0.99825000]
    fold  1:  [0.99900000]
    fold  2:  [0.99950000]
    fold  3:  [0.99850000]
    ----
    MEAN:     [0.99881250] + [0.00048007]
    FULL:     [0.99881250]

Accuracy score: [0.99800000]
Precision score: [0.99913570]
recall score: [0.99741156]
